The Music Ontology provides main concepts and properties fo describing music (i.e. artists, albums, tracks, but also performances, arrangements, etc.) on the Semantic Web

In [1]:
from rdflib import Graph

In [5]:
# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
g.parse("http://purl.org/ontology/mo/")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="turtle"))

Graph g has 2141 statements.
@prefix ao: <http://purl.org/ontology/ao/core#> .
@prefix bio: <http://purl.org/vocab/bio/0.1/> .
@prefix cc: <http://web.resource.org/cc/> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix frbr: <http://purl.org/vocab/frbr/core#> .
@prefix geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix keys: <http://purl.org/NET/c4dm/keys.owl#> .
@prefix mo: <http://purl.org/ontology/mo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix vs: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

mo:Arranger a owl:Class ;
    rdfs:label "arranger" ;

In [9]:
for s, p, o in g:
    print((s, p, o))

(rdflib.term.URIRef('http://purl.org/ontology/mo/trackNum'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#DatatypeProperty'))
(rdflib.term.BNode('nab5722cec9be484dabb92599e4e7da4bb57'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.URIRef('http://purl.org/ontology/mo/MusicArtist'))
(rdflib.term.URIRef('http://purl.org/ontology/mo/MusicalManifestation'), rdflib.term.URIRef('http://purl.org/ontology/mo/level'), rdflib.term.Literal('1'))
(rdflib.term.URIRef('http://purl.org/ontology/mo/PublishedLyrics'), rdflib.term.URIRef('http://www.w3.org/2003/06/sw-vocab-status/ns#term_status'), rdflib.term.Literal('stable'))
(rdflib.term.URIRef('http://purl.org/ontology/mo/SoloMusicArtist'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person'))
(rdflib.term.URIRef('http://purl.org/ontology/mo/MusicArtist'), rdflib.te